<a href="https://colab.research.google.com/github/Janina712/MLTSA22_JBoecher/blob/main/Other/3.%20MatchPhraseLength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Imports & Set-Up**

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned


In [ ]:
reading = pd.read_excel("reading_TextGrid_comb_BG_loop.xlsx")
interview = pd.read_excel("interview_TextGrid_comb_BG_loop.xlsx")

In [ ]:
IDs = ['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD']

**1. Assign Participant Group**

In [ ]:
group = pd.DataFrame(index = range(len(reading)),columns=["Group"])

In [ ]:
for i in range(0,len(reading)):
  if "_" in reading["ID"][i]:
    group["Group"][i] = "Control"
  else:
    group["Group"][i] = "PWS"

In [ ]:
reading = pd.concat([ group, reading], axis=1)

In [ ]:
reading = reading[reading.Type != "silence"]
reading.index = range(len(reading.index))

In [ ]:
reading.drop(['Unnamed: 0'], axis=1 , inplace = True)

In [ ]:
reading

,Group,ID,Type,Sound,Onset,Offset,Duration,Breath.Group,m
0,PWS,24fa,consonant,"""S""",0.143,0.263,0.120,1,33
1,PWS,24fa,vowel,"""OW1""",0.263,0.343,0.080,1,33
2,PWS,24fa,consonant,"""HH""",0.343,0.482,0.139,1,33
3,PWS,24fa,vowel,"""IH1""",0.482,0.573,0.091,1,33
4,PWS,24fa,consonant,"""R""",0.573,0.702,0.129,1,33
...,...,...,...,...,...,...,...,...,...
21478,Control,C13_RG,vowel,"""ER1""",87.691,87.881,0.190,28,38
21479,Control,C13_RG,consonant,"""N""",87.881,87.910,0.029,28,38
21480,Control,C13_RG,vowel,"""EY2""",87.910,88.081,0.171,28,38
21481,Control,C13_RG,consonant,"""M""",88.081,88.191,0.110,28,38


# **Part I: Pre-Matching Sound Counts**

**1. Vowels**

**1.1 Count Syllables Per Utterance**

In [ ]:
reading_vowels = reading[reading["Type"]  == "vowel"] 
reading_vowels.index = range(len(reading_vowels.index))

In [ ]:
syll_col = pd.DataFrame()  ## initialize group-level dataframe
for ID in IDs: ## loop over participnts
  syll_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
  subset_sounds = reading[reading["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_sounds.index = range(len(subset_sounds.index)) # reset index
  subset_vowels = subset_sounds[subset_sounds["Type"] == "vowel"]  # get subset of sound dataframe that corresponds to current participant
  subset_vowels.index = range(len(subset_vowels.index)) # reset index
  syll = subset_vowels["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
  syll.index = range(len(syll.index)) # reset index
  for a in range (0,len(syll)): # go through all breath groups that this participant produced
    syll_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
    syll_current_BG = pd.DataFrame(np.repeat(syll.iloc[a], syll.iloc[a], axis=0)) #replicate the sum sum times
    syll_current_ID = syll_current_ID.append([syll_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe 
  syll_col = syll_col.append([syll_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [ ]:
reading_vowels = pd.concat([reading_vowels, syll_col], axis=1)

In [ ]:
reading_vowels.rename(columns = {0:'Syllables'}, inplace = True) # rename new column

In [ ]:
reading_vowels.groupby("Group").mean()    ########### average counting 13 13 times

,Onset,Offset,Duration,Breath.Group,m,Syllables
Group,,,,,,
Control,38.778176,38.862372,0.084196,14.571033,40.731234,14.841310
PWS,34.920034,35.010382,0.090347,15.706039,32.564387,11.465808


**1.2 Average #Syllables By Participant**

In [ ]:
subj_col = pd.DataFrame(['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG'], columns = ["ID"])

In [ ]:
group_col = pd.DataFrame(index = range(len(subj_col)),columns=["Group"])
for i in range(0,len(group_col)):
  if "_" in subj_col["ID"][i]:
    group_col["Group"][i] = "Control"
  else:
    group_col["Group"][i] = "PWS"

In [ ]:
n = -1
avg_col = pd.DataFrame(index = range(len(subj_col)),columns=["Syllables"])
for ID in IDs: ## loop over participnts
  n = n + 1
  subset_BGs = reading_vowels[reading_vowels["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_BGs.index = range(len(subset_BGs.index)) # reset index
  BG_avg = subset_BGs.groupby("Breath.Group").mean()
  subj_avg = BG_avg["Syllables"].mean()
  avg_col["Syllables"][n] = subj_avg

In [ ]:
reading_vowels_Syll = pd.concat([group_col, subj_col, avg_col], axis=1)

In [ ]:
reading_vowels_Syll

,Group,ID,Syllables
0,PWS,24fa,8.423077
1,PWS,24ma,8.548387
2,PWS,24mb,8.432432
3,PWS,26f,10.666667
4,PWS,29ma,9.459459
5,PWS,29mb,8.3125
6,PWS,29mc,6.961538
7,PWS,32m,8.586207
8,PWS,34m,7.04
9,PWS,35mb,5.684211


**1.3 Compare Average Syllables per Utterance Across Groups**


In [ ]:
comparison_v = reading_vowels_Syll.groupby("Group").mean()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
comparison_v

,Syllables
Group,
Control,14.119860
PWS,8.539539


In [ ]:
control_v = comparison_v["Syllables"][0]
pws_v = comparison_v["Syllables"][1]
difference_v = control_v - pws_v

In [ ]:
string1 = (f"PWS produced on average {round(pws_v,2)} syllables per utterance, while control participants produced {round(control_v,2)} syllables on average.")
string2 = (f"This means that on average control participants produced {round(difference_v,2)} syllables more per utterance.")

In [ ]:
print(string1)
print(string2)

PWS produced on average 8.54 syllables per utterance, while control participants produced 14.12 syllables on average.
This means that on average control participants produced 5.58 syllables more per utterance.


**2. Consonants**

**2.1 Count Consonant Per Utterance**

In [ ]:
reading_cons = reading[reading["Type"]  == "consonant"] 
reading_cons.index = range(len(reading_cons.index))

In [ ]:
con_col = pd.DataFrame()  ## initialize group-level dataframe
for ID in IDs: ## loop over participnts
  con_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
  subset_sounds = reading[reading["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_sounds.index = range(len(subset_sounds.index)) # reset index
  subset_cons = subset_sounds[subset_sounds["Type"] == "consonant"]  # get subset of sound dataframe that corresponds to current participant
  subset_cons.index = range(len(subset_cons.index)) # reset index
  con = subset_cons["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
  con.index = range(len(con.index)) # reset index
  for a in range (0,len(con)): # go through all breath groups that this participant produced
    con_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
    con_current_BG = pd.DataFrame(np.repeat(con.iloc[a], con.iloc[a], axis=0)) #replicate the sum sum times
    con_current_ID = con_current_ID.append([con_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe 
  con_col = con_col.append([con_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [ ]:
reading_cons = pd.concat([reading_cons, con_col], axis=1)

In [ ]:
reading_cons.rename(columns = {0:'Consonants'}, inplace = True) # rename new column

In [ ]:
reading_cons

,Group,ID,Type,Sound,Onset,Offset,Duration,Breath.Group,m,Consonants
0,PWS,24fa,consonant,"""S""",0.143,0.263,0.120,1,33,14
1,PWS,24fa,consonant,"""HH""",0.343,0.482,0.139,1,33,14
2,PWS,24fa,consonant,"""R""",0.573,0.702,0.129,1,33,14
3,PWS,24fa,consonant,"""W""",0.702,0.732,0.030,1,33,14
4,PWS,24fa,consonant,"""R""",0.852,1.022,0.170,1,33,14
...,...,...,...,...,...,...,...,...,...,...
13004,Control,C13_RG,consonant,"""N""",87.251,87.371,0.120,28,38,20
13005,Control,C13_RG,consonant,"""S""",87.531,87.691,0.160,28,38,20
13006,Control,C13_RG,consonant,"""N""",87.881,87.910,0.029,28,38,20
13007,Control,C13_RG,consonant,"""M""",88.081,88.191,0.110,28,38,20


**2.2 Average #Consonants per Utterance**

In [ ]:
n = -1
avg_cons_col = pd.DataFrame(index = range(len(subj_col)),columns=["Consonants"])
for ID in IDs: ## loop over participnts
  n = n + 1
  subset_BGs = reading_cons[reading_cons["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_BGs.index = range(len(subset_BGs.index)) # reset index
  BG_avg = subset_BGs.groupby("Breath.Group").mean()
  subj_avg = BG_avg["Consonants"].mean()
  avg_cons_col["Consonants"][n] = subj_avg

In [ ]:
reading_cons_con = pd.concat([group_col, subj_col, avg_cons_col], axis=1)

In [ ]:
reading_cons_con 

,Group,ID,Consonants
0,PWS,24fa,13.269231
1,PWS,24ma,13.16129
2,PWS,24mb,12.756757
3,PWS,26f,16.740741
4,PWS,29ma,14.378378
5,PWS,29mb,13.125
6,PWS,29mc,10.692308
7,PWS,32m,13.344828
8,PWS,34m,10.84
9,PWS,35mb,8.368421


**2.3 Compare Average Consonants per Utterance Across Groups** 

In [ ]:
comparison_c = reading_cons_con.groupby("Group").mean()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
control_c = comparison_c["Consonants"][0]
pws_c = comparison_c["Consonants"][1]
difference_c = control_c - pws_c

In [ ]:
string3 = (f"PWS produced on average {round(pws_c,2)} consonants per utterance, while control participants produced {round(control_c,2)} consonants on average.")
string4 = (f"This means that on average control participants produced {round(difference_c,2)} consonants more per utterance.")

In [ ]:
print(string3)
print(string4)

PWS produced on average 13.13 consonants per utterance, while control participants produced 21.29 consonants on average.
This means that on average control participants produced 8.17 consonants more per utterance.


# **Part II: Match MLU Across Groups**

No straightforward approach. 
1. Cut the whole rest of the utterance after the -6th vowel. 
2. Cut vowels after 6th vowel and consonants after 9th consonant (rounded up)
--> Go with second 

**1. Vowels**

**1.1 Cut vowels from dataframe**

In [ ]:
control_IDs = ['C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG']

In [ ]:
cut_v = round(difference_v)

In [ ]:
syll_col_matched = pd.DataFrame()  ## initialize group-level dataframe
participant = pd.DataFrame()  ## initialize participant-level dataframe
reading_control = reading_vowels[reading_vowels["Group"]  == "Control"] 
reading_control.index = range(len(reading_control.index)) ## group
for ID in control_IDs:
  reading_control_ID = reading_control[reading_control["ID"]  == ID] 
  reading_control_ID.index = range(len(reading_control_ID.index))  ### person
  BGs = reading_control_ID["Breath.Group"].unique()
  for i in range(0,len(BGs)):
    reading_control_ID_BG = reading_control_ID[reading_control_ID["Breath.Group"]  == i] 
    reading_control_ID_BG.index = range(len(reading_control_ID_BG.index)) ## BG
    if len(reading_control_ID_BG) >= pws_v:
      reading_control_ID_BG.drop(reading_control_ID_BG.tail(cut_v).index, inplace = True) 
      participant = participant.append([reading_control_ID_BG], ignore_index = True)
    else:  
      participant = participant.append([reading_control_ID_BG], ignore_index = True)
syll_col_matched = syll_col_matched.append([participant], ignore_index = True) # add participant-level dataframe to group-level dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


**1.2 Count New Number of Vowels per Utterance**

In [ ]:
new_v_count = pd.DataFrame()  ## initialize group-level dataframe
for ID in control_IDs: ## loop over participnts
  new_v_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
  subset_sounds = syll_col_matched[syll_col_matched["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_sounds.index = range(len(subset_sounds.index)) # reset index
  new_v = subset_sounds["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
  new_v.index = range(len(new_v.index)) # reset index
  for a in range (0,len(new_v)): # go through all breath groups that this participant produced
    new_v_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
    new_v_current_BG = pd.DataFrame(np.repeat(new_v.iloc[a], new_v.iloc[a], axis=0)) #replicate the sum sum times
    new_v_current_ID = new_v_current_ID.append([new_v_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe 
  new_v_count = new_v_count.append([new_v_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [ ]:
syll_col_matched = pd.concat([syll_col_matched, new_v_count], axis=1)

In [ ]:
syll_col_matched.rename(columns = {0:'matched_V'}, inplace = True) # rename new column

In [ ]:
syll_col_matched

,Group,ID,Type,Sound,Onset,Offset,Duration,Breath.Group,m,Syllables,matched_V
0,Control,C1_DS,vowel,"""OW1""",0.384,0.544,0.160,1,86,31,22
1,Control,C1_DS,vowel,"""IH1""",0.674,0.804,0.130,1,86,31,22
2,Control,C1_DS,vowel,"""IY1""",0.965,1.095,0.130,1,86,31,22
3,Control,C1_DS,vowel,"""AA1""",1.095,1.385,0.290,1,86,31,22
4,Control,C1_DS,vowel,"""AH0""",1.474,1.585,0.111,1,86,31,22
...,...,...,...,...,...,...,...,...,...,...,...
2468,Control,C13_RG,vowel,"""AH1""",83.210,83.300,0.090,27,32,9,11
2469,Control,C13_RG,vowel,"""ER0""",83.350,83.440,0.090,27,32,9,11
2470,Control,C13_RG,vowel,"""AH0""",83.510,83.550,0.040,27,32,9,11
2471,Control,C13_RG,vowel,"""AE1""",83.670,83.770,0.100,27,32,9,11


In [ ]:
n = -1
control_subj_col = pd.DataFrame(index = range(len(control_IDs)),columns=["ID"])
avg_col = pd.DataFrame(index = range(len(control_IDs)),columns=["Syllables"])
for ID in control_IDs: ## loop over participnts
  n = n + 1
  subset_BGs = syll_col_matched[syll_col_matched["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_BGs.index = range(len(subset_BGs.index)) # reset index
  BG_avg = subset_BGs.groupby("Breath.Group").mean()
  subj_avg = BG_avg["matched_V"].mean()
  avg_col["Syllables"][n] = subj_avg
  control_subj_col["ID"][n] = ID

In [ ]:
matched_vowels = pd.concat([control_subj_col, avg_col], axis = 1)

In [ ]:
matched_vowels

,ID,Syllables
0,C1_DS,7.551724
1,C2_JH,7.304348
2,C3_MD,8.481481
3,C4_ZO,7.964286
4,C5_HL,7.375
5,C6_LH,8.222222
6,C7_SH,8.333333
7,C9_RD,7.25
8,C10_KS,7.655172
9,C11_NP,7.269231


**1.3 Compare Average Vowels per Utterance Across Groups**

In [ ]:
control_v_matched = matched_vowels["Syllables"].mean()

In [ ]:
difference_v_matched = control_v_matched - pws_v

In [ ]:
string5 = (f"After matching, PWS have on average {round(pws_v,2)} syllables per utterance, while control participants have {round(control_v_matched,2)} syllables on average.")
string6 = (f"This means that after matching the difference between control participants and PWS is reduced to {abs(round(difference_v_matched,2))} syllables per utterance.")

In [ ]:
print(string5)
print(string6)

After matching, PWS have on average 8.54 syllables per utterance, while control participants have 7.8 syllables on average.
This means that after matching the difference between control participants and PWS is reduced to 0.74 syllables per utterance.


**1.4 Test if the Remaining Difference is Statistically Significant**

To calculate averages, I count each breath group only once (rather than grouping the large df). This is because the large df has n rows per utterance, wich screws the average. 

In [ ]:
ttest_control = matched_vowels 

In [ ]:
ttest_pws = reading_vowels_Syll[0:18]      

In [ ]:
read_v_matched_test = stats.ttest_ind(ttest_control['Syllables'],
                      ttest_pws['Syllables'])

In [ ]:
if read_v_matched_test[1] >= 0.05:
  level = "insignificant"
else:
  level = "significant"

In [ ]:
string7 = (f"Control utterances that were longer than the mean average length of PWS utterances, were shortened by {round(cut_v,2)} syllables. \nAfter matching, PWS had {round(pws_v,2)} vowels per utterance, while control participants had {round(control_v_matched,2)} per utterance. \nThe difference is reduced to {abs(round(difference_v_matched,2))}, which is statistically {level} (p = {round(read_v_matched_test[1],2)}).")

In [ ]:
print(string7)

Control utterances that were longer than the mean average length of PWS utterances, were shortened by 6 syllables. 
After matching, PWS had 8.54 vowels per utterance, while control participants had 7.8 per utterance. 
The difference is reduced to 0.74, which is statistically insignificant (p = 0.1).


# **Part III: Save**

**1. Vowels**

**1.1 Create Output Dataframe**

In [ ]:
reading_vowels_PWS = reading_vowels[reading_vowels["Group"]  == "PWS"] 
reading_vowels_PWS.index = range(len(reading_vowels_PWS.index))

In [ ]:
syll_col_matched.drop(['Syllables'], axis=1 , inplace = True)
syll_col_matched.rename(columns = {'matched_V':'Syllables'}, inplace = True) # rename new column

In [ ]:
reading_vowels_matched = reading_vowels_PWS.append(syll_col_matched)

**1.2 Save as Excel**

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline


In [ ]:
dir = "3.MLU_Matched"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched


In [ ]:
reading_vowels_matched.to_excel("reading_TextGrid_comb_BG_vowelsMatched.xlsx")

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline
